<a href="https://colab.research.google.com/github/Nabarundutta/Machine-Learning/blob/main/mlday1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [15]:
!pip install pytesseract

In [6]:
!mkdir /mnt/data

In [8]:
import pandas as pd
import cv2
import numpy as np
import pytesseract
import requests
import gc

In [9]:
# Function to download and process the image from a URL
def get_image_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if request is successful
        img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        return img
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")
        return None

In [10]:
# Function to extract text from an image using pytesseract
def extract_text_from_image(img):
    try:
        # Convert image to grayscale for better OCR results
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Perform OCR to extract text
        text = pytesseract.image_to_string(gray_img)
        return text.strip()  # Return the extracted text
    except Exception as e:
        print(f"Error during OCR: {e}")
        return ""

In [11]:
# Function to process a batch of the DataFrame, retrieve images, extract text, and save results
def process_batch(df_chunk):
    # Apply image downloading and text extraction for each URL in the chunk
    df_chunk['text'] = df_chunk['image_link'].apply(lambda url: extract_text_from_image(get_image_from_url(url)))
    return df_chunk


In [12]:
# Main function to process the CSV file in chunks
def process_csv_in_batches(csv_path, output_path, chunksize=100):
    # Initialize a variable to track if the header has been written
    header_written = False

    # Process the CSV file in batches
    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        # Process each batch to download images and extract text
        processed_chunk = process_batch(chunk)

        # Save the processed chunk with text column to the output CSV
        processed_chunk.to_csv(output_path, mode='a', header=not header_written, index=False)
        header_written = True  # Write the header only once, for the first chunk

        # Free up memory by forcing garbage collection
        del processed_chunk
        gc.collect()

    print("Processing complete, and data saved to", output_path)


In [13]:
csv_path = 'train.csv'  # Path to your input CSV file
output_path = 'processed_train_with_text.csv'  # Path to save the processed data

In [1]:
!sudo apt update
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,030 kB]
Get:14 http://securi

In [ ]:
# Process the CSV file in batches, with a defined batch size
process_csv_in_batches(csv_path, output_path, chunksize=100)